In [18]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Model 
from keras.layers import Input, Dense, ReLU, Activation, Conv1D, Normalization, LayerNormalization, BatchNormalization 


In [19]:
import numpy as np
import random
class Powerlayer(tf.keras.layers.Layer):
  def __init__(self):
    super(Customdense, self).__init__()
    self.normal = []
  def build(self, input):
    self.normal = np.var(input)+np.mean(input)*np.mean(input)
  def call(self, input):
    y = input/self.normal
    return y

In [20]:
from keras import layers
from keras import backend
from keras.engine import base_layer
from keras.utils import tf_utils
import tensorflow as tf
from tensorflow.python.util.tf_export import keras_export
@keras_export("keras.activations.powernorm")
@tf.__internal__.dispatch.add_dispatch_support
def powernorm(x, axis=-1):
    if x.shape.rank > 1:
        if isinstance(axis, int):
            output = tf.nn.powernorm(x, axis=axis)
        else:
            # nn.softmax does not support tuple axis.
            e = x
            s = tf.reduce_sum(x*x, axis=axis, keepdims=True)/len(x)
            output = e / s
    else:
        raise ValueError(
            "Cannot apply powernorm to a tensor that is 1D. "
            f"Received input: {x}"
        )

    # Cache the logits to use for crossentropy loss.
    output._keras_logits = x
    return output

@keras_export('keras.layers.Fsolayer')
class Fsolayer(base_layer.BaseRandomLayer):

  def __init__(self, **kwargs):
    super(Fsolayer, self).__init__( **kwargs)
    self.supports_masking = True

  def call(self, inputs, training=None):

    def noised():
      J_S_D=0.3     #%%% Jetter Standard Deviation
      H_l=0.8      #%%%% pass loss factor
      a=0.1        #%%% Radius
      R=0.5         #%%% responsitivity
      L=2000        #%%%% distance in m
      theta=2.5/1000 #%%% divergence angle
      W_z=theta*L     #%%% Beam radius
      wl=1550/1000000000   #%%%% wavelength in nm	
      ha = []
      alpha = 4.505303125003634
      beta  = 2.714091563350723
      elements = 1	
      temp1 = random.gammavariate(alpha, 1/alpha)
      temp2 = random.gammavariate(beta, 1/beta)
      ha = temp1*temp2
      W_z_eq = 5.001047351093471
      A_o = 6.774000000000000e-04
      r_pdf = np.random.rayleigh(J_S_D,elements)
      r_pdf = list(r_pdf)
      hp = A_o*np.exp(-np.multiply(r_pdf,r_pdf)*2/W_z_eq)
      return inputs*ha*hp

    return backend.in_train_phase(noised, inputs, training=training)

  def get_config(self):
    config = {}
    base_config = super(Fsolayer, self).get_config()
    return dict(list(base_config.items()) + list(config.items()))

  @tf_utils.shape_type_conversion
  def compute_output_shape(self, input_shape):
    return input_shape

In [21]:
import numpy as np
import random
#Generate random numbers between 0 and 15 for training data
symbols = np.random.randint(0,16,100000)
X_train = np.zeros((symbols.size, symbols.max()+1))
X_train[np.arange(symbols.size),symbols] = 1
#X_train.shape

In [22]:
#Generate random numbers between 0 and 15 for test data
symbols = np.random.randint(0,16,10000)
X_test = np.zeros((symbols.size, symbols.max()+1))
X_test[np.arange(symbols.size),symbols] = 1
X_test.shape

(10000, 16)

In [23]:
##################### Noise and Power
R=4/7;
Eb_No_training=7; ## dB
Eb_No_training_ratio=10**(Eb_No_training/10);
stddev=(2*R*Eb_No_training_ratio)**(-0.5);
N_S_D=1e-07; 
## Power
Pdb = np.linspace(-25,25,51);   #dbm
P_m = 10**(Pdb/10);   #mW
P = P_m/1000;         #W

In [24]:
##################### Autoencoder Parts
inputs= X_test.shape[1]
#define the encoder 
encoder_input = Input(shape=(inputs,))
encoder_layer = Dense(inputs, activation="relu")(encoder_input)
encoder_layer = Dense(7, activation="linear")(encoder_layer)
encoder_output = BatchNormalization()(encoder_layer)
# FSO channel
fso=Fsolayer()(encoder_output)
channel_output=tf.keras.layers.GaussianNoise(N_S_D)(fso) 
encoder_input.shape

TensorShape([None, 16])

In [25]:
# define the decoder 
decoder= Dense(inputs, activation="relu")(channel_output)
from tensorflow import expand_dims, squeeze
layer = expand_dims(decoder, 2)
decoder = Conv1D(filters=16, kernel_size=1,
                          strides=1, padding='valid',
                          data_format='channels_first')(layer)
layer2 = squeeze(decoder,[2])    ## to remove the last dim                      
decoder_output = Dense(inputs, activation="softmax")(layer2)

# define the autoencoder

autoencoder=Model(inputs=encoder_input, outputs=decoder_output)
decoder_output.shape

TensorShape([None, 16])

In [26]:
## encoder side
encoder=Model(inputs=encoder_input, outputs=encoder_output)
#decoder side 
decoder=Model(inputs=channel_output, outputs=decoder_output)

In [27]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [29]:
tf.test.is_gpu_available()

False

In [28]:
autoencoder.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
history=autoencoder.fit(X_train,X_train,epochs=10, validation_data=(X_test,X_test),batch_size=1000)
#autoencoder.save("functional_encoder.h5")

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'gradient_tape/model_3/conv1d_1/Conv1D/Conv2DBackpropInput' defined at (most recent call last):
    File "c:\Users\moata\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\moata\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\moata\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\moata\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "c:\Users\moata\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\moata\AppData\Local\Programs\Python\Python310\lib\

In [ ]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 16)]              0         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 7)                 119       
                                                                 
 batch_normalization (BatchN  (None, 7)                28        
 ormalization)                                                   
                                                                 
 fsolayer (Fsolayer)         (None, 7)                 0         
                                                                 
 gaussian_noise (GaussianNoi  (None, 7)                0         
 se)                                                         

In [ ]:
encoder.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 16)]              0         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 7)                 119       
                                                                 
 batch_normalization (BatchN  (None, 7)                28        
 ormalization)                                                   
                                                                 
Total params: 419
Trainable params: 405
Non-trainable params: 14
_________________________________________________________________


In [ ]:
decoder.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 7)]               0         
                                                                 
 dense_2 (Dense)             (None, 16)                128       
                                                                 
 tf.expand_dims (TFOpLambda)  (None, 16, 1)            0         
                                                                 
 conv1d (Conv1D)             (None, 16, 1)             272       
                                                                 
 tf.compat.v1.squeeze (TFOpL  (None, 16)               0         
 ambda)                                                          
                                                                 
 dense_3 (Dense)             (None, 16)                272       
                                                           

In [ ]:
import numpy as np
encodedsymbols=encoder.predict(X_test)
# noise = np.random.normal(0,stddev,[10000,7])
# noisy=encodedsymbols+noise
# decoded_symbols=decoder.predict(noisy)
# #convert the probability to hot vector
# a=decoded_symbols
# idx = np.argmax(a, axis=-1)
# a = np.zeros( a.shape )
# a[ np.arange(a.shape[0]), idx] = 1
rms = np.sqrt(np.mean(encodedsymbols**2))
rms*rms

313/313 [==============================] - 1s 999us/step


0.053214964

In [ ]:
######==============================>plotting
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
import matplotlib.pyplot as plt
epochs = range(1, len(acc)+1)
plt.plot(epochs, acc, 'k--', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'k--', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

NameError: name 'history' is not defined

In [ ]:
#Generate random numbers between 0 and 15 for BER calculations
test_size=1000000;
symbols = np.random.randint(0,16,test_size)
X_test = np.zeros((symbols.size, symbols.max()+1))
X_test[np.arange(symbols.size),symbols] = 1

#generating a E/N vector
R=4/7;
num_steps=26
Eb_No=np.linspace(-4,8,num_steps); ## dB

In [ ]:
BER=np.zeros(num_steps)
for i in range(len(Eb_No)):
  Eb_No_ratio=10**(Eb_No[i]/10);
  stddev=(2*R*Eb_No_ratio)**(-0.5);
  encodedsymbols=encoder.predict(X_test)
  noise = np.random.normal(0,stddev,[test_size,7])
  noisy=encodedsymbols+noise
  a=decoder.predict(noisy)
  idx = np.argmax(a, axis=-1)
  a = np.zeros( a.shape )
  a[ np.arange(a.shape[0]), idx] = 1
  recovered_symbols=np.argmax(a, axis=1)
  errors=np.count_nonzero(recovered_symbols-symbols)
  BER[i]=errors/len(symbols)



In [ ]:
import matplotlib.pyplot as plt

plt.semilogy(Eb_No, BER, 'k--')
plt.title('Block Error Rate')
plt.xlabel('E_b/N_o')
plt.grid(True)
plt.show()

In [ ]:
class Customdense(tf.keras.layers.Layer):
  def __init__(self, num_units, activation = "relu"):
    super(Customdense, self).__init__()
    self.num_units = num_units
    self.activation = Activation(activation)
  def build(self, input_shape):
    self.weight = self.add_weight(shape = [input_shape[-1], self.num_units])
    self.bias = self.add_weight(shape =[self.num_units])
  def call(self, input):
    y = tf.matmul(input, self.weight) + self.bias
    y = self.activation(y)
    return y

In [ ]:
import tensorflow as tf
x = tf.constant([5, 1, 2, 4])
print(tf.reduce_max(x))


tf.Tensor(5, shape=(), dtype=int32)


In [ ]:
from keras import layers
from keras import backend
from keras.engine import base_layer
from keras.utils import tf_utils
import tensorflow.compat.v2 as tf
from tensorflow.python.util.tf_export import keras_export
@keras_export("keras.activations.powernorm")
@tf.__internal__.dispatch.add_dispatch_support
def powernorm(x, axis=-1):
    if x.shape.rank > 1:
        if isinstance(axis, int):
            output = tf.nn.powernorm(x, axis=axis)
        else:
            # nn.softmax does not support tuple axis.
            e = x
            s = tf.reduce_sum(x*x, axis=axis, keepdims=True)/len(x)
            output = e / s
    else:
        raise ValueError(
            "Cannot apply softmax to a tensor that is 1D. "
            f"Received input: {x}"
        )

    # Cache the logits to use for crossentropy loss.
    output._keras_logits = x
    return output